In [1]:
import requests
import torch
import sympy
from sympy import symbols, QQ
import re
import time
import warnings
import simulated_bifurcation as sb

In [2]:
def preprocess(model, gens):
    """
    Establish the correct/allowed model symbols (gens) and update the model accordingly
    """
    if gens is None:
        gens = []
        for gen in model.ring.gens:
            if re.match('x[1-9][0-9]*', str(gen)):
                gens.append(gen)
            # elif re.match('x0$', str(gen)):
            #     pass                
        gens = tuple(gens)

    drop_gens = set(model.ring.gens).difference(set(gens))
    for gen in drop_gens:
        warnings.warn(f'Symbol "{str(gen)}" was dropped from the model')
        model = model.drop(model.ring(gen))

    return model, gens

def split_constant_term(model, gens):
    """
    Separate the constant term from the polynomial expression
    """
    model_dict = model.to_dict()
    constant_monom = (0,) * len(gens)
    constant = model_dict.pop(constant_monom, None)
    if constant is None:
        constant = 0
    else:
        constant = float(constant)
        if constant.is_integer():
            constant = int(constant)

    return model.ring(model_dict), constant


def square_diagonal_terms(model, gens):
    """
    Convert linear terms, $x_i$, to squared terms, $x_i*x_i$ 
    """
    tmp_monom = [0,] * len(gens)
    squared_dict = model.to_dict().copy()
    for i, gen in enumerate(gens):
        tmp_monom[i] = 1
        coeff = squared_dict.pop(tuple(tmp_monom), None)
        tmp_monom[i] = 2
        if coeff is not None:
            squared_dict[tuple(tmp_monom)] = squared_dict.get(tuple(tmp_monom), 0) + coeff
        tmp_monom[i] = 0

    return model.ring(squared_dict)


def poly_to_Q(model, gens=None):
    """
    Convert a `sympy` PolyElement model to its corresponding Q matrix and constant
    """

    model, gens = preprocess(model, gens)
    model, constant = split_constant_term(model, gens)
    model = square_diagonal_terms(model, gens)

    Q = torch.zeros((len(gens), len(gens)), dtype=torch.float64)
    monoms = torch.tensor(model.monoms(), dtype=torch.int32)
    coeffs = torch.tensor(model.coeffs(), dtype=torch.float64)
    for i in range(len(monoms)):
        idx = torch.nonzero(monoms[i])
        if len(idx) == 2:
            value = 0.5*coeffs[i]
            Q[idx[0], idx[1]] = value
            Q[idx[1], idx[0]] = value
        elif len(idx) == 1:
            Q[idx[0], idx[0]] = coeffs[i]
        else:
            warnings.warn(f'Unrecognized non-quadratic terms were excluded from the model')
    
    return Q, constant

See:

* [Portfolio Optimization Example](https://www.mathworks.com/help/optim/ug/quadratic-programming-portfolio-optimization-problem-based.html)
* [Portfolio Data](http://people.brunel.ac.uk/~mastjjb/jeb/orlib/portinfo.html)
* [port5](http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/port5.txt)
* [Original Paper](https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=08ed9a39b32fc84a5a3ad7325dfcf6ecfe19183e)

In [3]:
url = "http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/port5.txt"
response = requests.get(url)

According to the [original source](http://people.brunel.ac.uk/~mastjjb/jeb/orlib/portinfo.html), the format of the data is:

```
number of assets (N)
for each asset i (i=1,...,N):
   mean return, standard deviation of return
for all possible pairs of assets:
   i, j, correlation between asset i and asset j
```

Note that the correlation is only the upper triangular matrix

In [4]:
lines = (l.strip() for l in response.text.split('\n'))
n_assets = int(next(lines))

mean_return = torch.empty(n_assets, dtype=torch.float64)
stddev_return = torch.empty(n_assets, dtype=torch.float64)
for i in range(n_assets):
    mean_return[i], stddev_return[i] = (float(l.strip()) for l in next(lines).split())

corr = torch.full((n_assets, n_assets), torch.nan, dtype=torch.float64)
for l in lines:
    if l != "":
        i, j, value = l.split()
        i, j, value = int(i)-1, int(j)-1, float(value)
        corr[i, j], corr[j, i] = value, value

In [5]:
covar = corr * (stddev_return * stddev_return)

In [6]:
covar

tensor([[0.0014, 0.0010, 0.0012,  ..., 0.0006, 0.0007, 0.0004],
        [0.0006, 0.0025, 0.0009,  ..., 0.0005, 0.0007, 0.0002],
        [0.0008, 0.0010, 0.0023,  ..., 0.0005, 0.0007, 0.0003],
        ...,
        [0.0007, 0.0010, 0.0010,  ..., 0.0012, 0.0009, 0.0003],
        [0.0007, 0.0011, 0.0010,  ..., 0.0007, 0.0015, 0.0003],
        [0.0007, 0.0007, 0.0010,  ..., 0.0005, 0.0006, 0.0008]],
       dtype=torch.float64)

In [7]:
r = 0.002

In [8]:
Q = 0.5 * covar

In [9]:
x, Peq = sympy.symbols(f'x(:{n_assets+1})'), sympy.symbols('Peq')
# polynomial ring of symbolic coefficients
R = QQ[x[1:]+(Peq,)]

In [10]:
start = time.time()
equality = -1
for i in range(1, n_assets+1):
    equality += x[i]
    
penalties = R.from_sympy(Peq*(equality)**2)
print(time.time() - start)

0.592526912689209


In [11]:
start = time.time()
# Q, constant = poly_to_Q(penalties.evaluate(R(Peq), 10))
penalties, constant = poly_to_Q(penalties.subs(R(Peq), 10))
print(time.time() - start)

/var/folders/cc/95f7218n54l6mb7jmsyzvssr0000gn/T/ipykernel_33049/1937982928.py:16: UserWarning: Symbol "Peq" was dropped from the model
  warnings.warn(f'Symbol "{str(gen)}" was dropped from the model')


0.7910847663879395


In [12]:
sb.maximize(Q+penalties, input_type='binary', ballistic=True)

Bifurcated agents: 100%|█████████████████████| 128/128 [00:00<00:00, 436.91it/s]


(tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1.,

In [13]:
mean_return

tensor([-1.1170e-03,  3.1230e-03, -3.4710e-03, -5.8500e-04, -7.6900e-04,
        -3.1240e-03, -3.2500e-03, -2.1740e-03,  3.7300e-03, -2.4500e-03,
        -9.5000e-04, -3.7000e-03, -1.1220e-03, -2.4410e-03, -1.7060e-03,
        -4.6150e-03, -3.7740e-03, -7.0400e-04, -1.8720e-03, -3.5470e-03,
        -1.9090e-03, -1.5190e-03, -1.7700e-03, -2.9700e-03, -2.9090e-03,
        -9.1300e-04, -3.1660e-03, -1.5730e-03, -4.0370e-03, -2.0490e-03,
        -1.9340e-03, -2.4000e-05, -2.6850e-03, -4.8000e-05,  7.8000e-05,
         7.2700e-04, -1.8400e-04,  2.6700e-04, -1.6310e-03,  3.0830e-03,
        -7.2100e-04, -5.0500e-04,  3.3890e-03, -3.6880e-03, -2.1930e-03,
        -1.2670e-03, -1.5120e-03, -1.8500e-04, -2.7840e-03,  5.9900e-04,
        -1.0450e-03, -1.8680e-03, -2.6300e-03, -3.2350e-03, -4.2010e-03,
        -1.1520e-03, -8.4890e-03, -3.3300e-04, -1.0630e-03,  4.7000e-05,
         1.1170e-03,  3.3070e-03, -3.2220e-03, -4.7800e-03, -2.8300e-03,
        -2.6500e-04, -1.1530e-03, -4.4600e-04, -4.2